In [128]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import seaborn as sns

In [78]:
# run matlab code to find regions that intersect with the lesion

# overlap_fs_lesion.m


'/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/cortical_atrophy/data/FREESURFER_ENIGMA/overlap_roi_lesion/sub-r010s012_pct_overlap_fs_lesion.txt'

In [154]:
counter =0
sub_names=[]
thr = 0
for file in glob.glob("/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/cortical_atrophy/data/FREESURFER_ENIGMA/overlap_roi_lesion/*.txt"):
    sub_name = str(file[112:124])
    if counter == 0:
        base = list(pd.read_table(file, header=0,delimiter=',').columns)

    values = pd.read_table(file,header=0, delimiter =',')
    values_bin = values > thr

    values.insert(0,'Subject', sub_name )
    values_bin.insert(0,'Subject', sub_name )

    if counter == 0:
        all_overlap_pct = values
        all_overlap_pct_bin = values_bin

        counter = counter +1
        continue
    else:
        all_overlap_pct=pd.concat((all_overlap_pct, values))
        all_overlap_bin=pd.concat((all_overlap_pct_bin, values_bin))


all_overlap_pct.sort_values(by='Subject',inplace=True)
all_overlap_bin.sort_values(by='Subject',inplace=True)

all_overlap_pct.to_csv('/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/cortical_atrophy/data/FREESURFER_ENIGMA/overlap_roi_lesion/overlap_pct_allrois_allENIGMA_subjects.csv')
all_overlap_bin.to_csv('/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/cortical_atrophy/data/FREESURFER_ENIGMA/overlap_roi_lesion/overlap_bin_allrois_allENIGMA_subjects.csv')

In [272]:
enigma_combat_CT =  pd.read_table('/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/cortical_atrophy/data/FREESURFER_ENIGMA/enigma_thickness_datasetvar_COMBAT.csv', header=0, sep=',')

mean_HCP_combat_CT =  pd.read_table('/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/cortical_atrophy/data/combat_CT_HCP_AGING_YA_age_mean.csv', header=0, sep=',')
std_HCP_combat_CT =  pd.read_table('/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/cortical_atrophy/data/combat_CT_HCP_AGING_YA_age_std.csv', header=0, sep=',')

age_enigma=enigma_combat_CT['age']

columns=[]
for col in enigma_combat_CT.columns:
    if col == 'Subject':
        continue
    col=col.replace('_thickness', '')
    col=col.replace('_', '-') 
    col = 'ctx-'+ col
    columns.append(col)

columns.remove('ctx-rh-MeanThickness')
columns.remove('ctx-lh-MeanThickness')
columns.remove('ctx-age')
columns.remove('ctx-sex')
columns.remove('ctx-dataset')
columns.remove('ctx-site')

z_enigma_df = pd.DataFrame(columns=columns)
z_enigma_df_NaNd = pd.DataFrame(columns=columns)

for s in range(0, len(age_enigma)):
    subj_age = age_enigma.iloc[s]
    subject_id = enigma_combat_CT.iloc[s, 0]
    refage_sub = np.round(subj_age)
    hcp_mean_age_vector = mean_HCP_combat_CT[mean_HCP_combat_CT['age']==refage_sub]
    hcp_std_age_vector = std_HCP_combat_CT[std_HCP_combat_CT['age']==refage_sub]

    alloverlaps = all_overlap_pct[all_overlap_pct['Subject']==subject_id]
    ct_enigma_sub = enigma_combat_CT[enigma_combat_CT['Subject']==subject_id]
    z_enigma_df_NaNd.loc[s,'Subject']=subject_id

    for col in columns:
        if alloverlaps[col].any()==0:
            col2=col.replace('-', '_') 
            col2=col2.replace('ctx_', '')

            mean_roi = hcp_mean_age_vector[col2+'_thickness_mean']
            std_roi = hcp_std_age_vector[col2+'_thickness_std']
            ct_roi = ct_enigma_sub[col2+'_thickness']
            z_enigma = (ct_roi.values-mean_roi.values)/std_roi.values
            z_enigma_df_NaNd.loc[s,col]=float(z_enigma)
        else:
            z_enigma_df_NaNd.loc[s,col]=float(0.0000)

z_enigma_df.to_csv('/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/cortical_atrophy/data/FREESURFER_ENIGMA/enigma_combat_thickness_zscore.csv', header=True, sep=',')
z_enigma_df_NaNd.to_csv('/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/cortical_atrophy/data/FREESURFER_ENIGMA/enigma_combat_thickness_zscore_NANs.csv', header=True, sep=',')

In [133]:
# make figures of zcsores  (better viz in R)

fig, axs = plt.subplots(ncols=5, nrows=7,figsize=(25,35))
counter =0
rowcount=0
# left hem

for i in range(2,37):
    y_zsc = z_enigma_df.iloc[:,i-1]
    sns.histplot(x=y_zsc, data=z_enigma_df, ax=axs[rowcount,counter%5],palette="hls")

    #axs[rowcount,counter%5].set_ylim([1.2, 5])
    #axs[rowcount,counter%5].set_ylim([1.2, 5])
    axs[rowcount,counter%5].tick_params(axis='x', labelsize=10)
    axs[rowcount,counter%5].tick_params(axis='y', labelsize=10)
   # axs[rowcount,counter%5].legend(['ComBat-adjusted HCP mean',' +/- 1 standard deviation','ComBat-Adjusted ENIGMA'])

   # axs[rowcount,counter%5].get_legend().remove()
    counter = counter +1
    if (counter%5 ==0) & (counter!=0):
        rowcount = rowcount+1

#plt.savefig('/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/cortical_atrophy/figures/lefthem_CT_hcp_YA_AGING_ENIGMA_meanstd.jpeg',bbox_inches='tight')
plt.savefig('/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/cortical_atrophy/figures/lh_zscores_ENIGMA.pdf',bbox_inches='tight')